In [1]:
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [3,972 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,773 kB]
Get:13 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran

In [2]:
!wget -q https://archive.apache.org/dist/spark/spark-3.5.5/spark-3.5.5-bin-hadoop3.tgz
!tar -xzf spark-3.5.5-bin-hadoop3.tgz
!mv spark-3.5.5-bin-hadoop3 /content/spark

In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark"
os.environ["PATH"] += ":/content/spark/bin"

In [4]:
!pip install -q findspark

In [5]:
import findspark
findspark.init()

In [6]:
from pyspark import SparkContext, SparkConf
import pyspark.sql as sql
from pyspark.sql import SparkSession
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import udf, col, max, sum, countDistinct
from pyspark.sql.functions import radians, cos, sin, sqrt, asin

spark = SparkSession.builder\
        .master("local")\
        .appName("LR2")\
        .config("spark.jars.packages", "com.databricks:spark-xml_2.12:0.15.0") \
        .getOrCreate()

sc = spark.sparkContext

print(sc.version)

3.5.5


In [7]:
from google.colab import files
uploaded1 = files.upload()
uploaded2 = files.upload()

Saving programming-languages.csv to programming-languages.csv


Saving posts_sample.xml to posts_sample.xml


In [8]:
languages = spark.read.csv("programming-languages.csv", header=True, inferSchema=True)
posts = spark.read.format("xml").option("rowTag", "row").load("posts_sample.xml")

languages.printSchema()
languages.show(5, truncate=False)


posts.printSchema()
posts.show(5, truncate=False)


root
 |-- name: string (nullable = true)
 |-- wikipedia_url: string (nullable = true)

+----------+---------------------------------------------------------+
|name      |wikipedia_url                                            |
+----------+---------------------------------------------------------+
|A# .NET   |https://en.wikipedia.org/wiki/A_Sharp_(.NET)             |
|A# (Axiom)|https://en.wikipedia.org/wiki/A_Sharp_(Axiom)            |
|A-0 System|https://en.wikipedia.org/wiki/A-0_System                 |
|A+        |https://en.wikipedia.org/wiki/A%2B_(programming_language)|
|A++       |https://en.wikipedia.org/wiki/A%2B%2B                    |
+----------+---------------------------------------------------------+
only showing top 5 rows

root
 |-- _AcceptedAnswerId: long (nullable = true)
 |-- _AnswerCount: long (nullable = true)
 |-- _Body: string (nullable = true)
 |-- _ClosedDate: timestamp (nullable = true)
 |-- _CommentCount: long (nullable = true)
 |-- _CommunityOwnedDate: tim

**Сформировать отчёт с информацией о 10 наиболее популярных языках программирования по итогам года за период с 2010 по 2020 годы. Отчёт будет отражать динамику изменения популярности языков программирования и представлять собой набор таблиц "топ-10" для каждого года.**

Получившийся отчёт сохранить в формате Apache Parquet.

In [9]:
# Оставляем только нужные столбцы: _CreationDate и _Tags
posts = posts.select("_CreationDate", "_Tags")
print("Посты после удаления лишних столбцов:")
posts.show(15, truncate=False)

Посты после удаления лишних столбцов:
+-----------------------+------------------------------------------------------+
|_CreationDate          |_Tags                                                 |
+-----------------------+------------------------------------------------------+
|2008-07-31 21:42:52.667|<c#><floating-point><type-conversion><double><decimal>|
|2008-07-31 22:08:08.62 |<html><css><internet-explorer-7>                      |
|2008-07-31 22:17:57.883|NULL                                                  |
|2008-07-31 23:40:59.743|<c#><.net><datetime>                                  |
|2008-07-31 23:55:37.967|<c#><datetime><time><datediff><relative-time-span>    |
|2008-07-31 23:56:41.303|NULL                                                  |
|2008-08-01 00:42:38.903|<html><browser><timezone><user-agent><timezone-offset>|
|2008-08-01 00:59:11.177|<.net><math>                                          |
|2010-09-20 16:18:14.58 |NULL                                          

In [47]:
from pyspark.sql.functions import year, split, regexp_replace, explode, lower, regexp_extract, trim

In [48]:
# Извлекаем год из _CreationDate и оставляем данные за 2010–2020 годы
posts_filtered = posts.withColumn(
    "Year",
    year(col("_CreationDate").cast("timestamp"))
).filter((col("Year") >= 2010) & (col("Year") <= 2020))
print("Посты после фильтрации по годам (2010–2020):")
posts_filtered.show(10, truncate=False)

Посты после фильтрации по годам (2010–2020):
+-----------------------+-----+----+
|_CreationDate          |_Tags|Year|
+-----------------------+-----+----+
|2010-09-20 16:18:14.58 |NULL |2010|
|2010-09-20 18:36:28.023|NULL |2010|
|2010-09-20 19:04:13.587|NULL |2010|
|2010-09-21 02:07:51.1  |NULL |2010|
|2010-09-21 07:33:58.707|NULL |2010|
|2010-09-21 11:46:48.24 |NULL |2010|
|2010-09-21 12:07:58.707|NULL |2010|
|2010-09-21 13:16:27.327|NULL |2010|
|2010-09-21 15:09:55.67 |NULL |2010|
|2010-09-21 16:22:24.963|NULL |2010|
+-----------------------+-----+----+
only showing top 10 rows



In [57]:
# Извлекаем теги из колонки _Tags
posts_with_tags = posts_filtered.withColumn(
    "TagArray",
    split(regexp_replace(col("_Tags"), r"[<>]", " "), " ")
)
print("Посты после обработки тегов (разделение на массивы):")
posts_with_tags.select("Year", "TagArray").show(100, truncate=False)

Посты после обработки тегов (разделение на массивы):
+----+---------------------------------------------------------------------------+
|Year|TagArray                                                                   |
+----+---------------------------------------------------------------------------+
|2010|NULL                                                                       |
|2010|NULL                                                                       |
|2010|NULL                                                                       |
|2010|NULL                                                                       |
|2010|NULL                                                                       |
|2010|NULL                                                                       |
|2010|NULL                                                                       |
|2010|NULL                                                                       |
|2010|NULL                        

In [56]:
# Разворачиваем массив тегов в строки (explode) и убираем пустые строки
posts_exploded = posts_with_tags.select(
    col("Year"),
    explode(col("TagArray")).alias("Tag")
).filter(col("Tag") != "")
print("Посты после explode (каждый тег в отдельной строке):")
posts_exploded.show(80, truncate=False)

Посты после explode (каждый тег в отдельной строке):
+----+------------------+
|Year|Tag               |
+----+------------------+
|2010|c++               |
|2010|character-encoding|
|2010|sharepoint        |
|2010|infopath          |
|2010|iphone            |
|2010|app-store         |
|2010|in-app-purchase   |
|2010|symfony1          |
|2010|schema            |
|2010|doctrine          |
|2010|fixtures          |
|2010|java              |
|2010|visual-studio-2010|
|2010|stylecop          |
|2010|cakephp           |
|2010|file-upload       |
|2010|swfupload         |
|2010|git               |
|2010|cygwin            |
|2010|putty             |
|2010|drupal            |
|2010|drupal-6          |
|2010|php               |
|2010|wordpress         |
|2010|memory            |
|2010|c#                |
|2010|winforms          |
|2010|data-binding      |
|2010|checkedlistbox    |
|2010|c#                |
|2010|asp.net           |
|2010|exception-handling|
|2010|sql               |
|2010|xml  

In [58]:
languages_prepared = languages.select(
    lower(regexp_extract(col("name"), r"^([^–]+)", 1)).alias("lang_clean"),
    col("name").alias("lang_original")
)

# Сбор списка корректных языков
valid_languages = languages_prepared.select("lang_clean").rdd.flatMap(lambda x: x).collect()

print("Корректные языки программирования (первые 100):")
print(valid_languages[:100])

Корректные языки программирования (первые 100):
['a# .net', 'a# (axiom)', 'a-0 system', 'a+', 'a++', 'abap', 'abc', 'abc algol', 'abset', 'absys', 'acc', 'accent', 'ace dasl', 'acl2', 'act-iii', 'action!', 'actionscript', 'ada', 'adenine', 'agda', 'agilent vee', 'agora', 'aimms', 'alef', 'alf', 'algol 58', 'algol 60', 'algol 68', 'algol w', 'alice', 'alma-0', 'ambienttalk', 'amiga e', 'amos', 'ampl', 'apex (salesforce.com)', 'apl', "app inventor for android's visual block language", 'applescript', 'arc', 'arexx', 'argus', 'aspectj', 'assembly language', 'ats', 'ateji px', 'autohotkey', 'autocoder', 'autoit', 'autolisp / visual lisp', 'averest', 'awk', 'axum', 'b', 'babbage', 'bash', 'basic', 'bc', 'bcpl', 'beanshell', 'batch (windows/dos)', 'bertrand', 'beta', 'bigwig', 'bistro', 'bitc', 'bliss', 'blockly', 'bloop', 'blue', 'boo', 'boomerang', 'bourne shell (including', 'bash and', 'ksh )', 'brew', 'bpel', 'c', 'c--', 'c++ ', 'c# ', 'c/al', 'caché objectscript', 'c shell', 'caml', 'cay

In [59]:
languages_prepared = languages.select(
    trim(
        lower(
            # Сначала вырезаем часть, которая идёт после "–"
            regexp_replace(
                # Затем вырезаем часть, которая идёт после "("
                regexp_replace(col("name"), r"–.*", ""),
                r"\(.*", ""
            )
        )
    ).alias("lang_clean"),
    col("name").alias("lang_original")
)

# Сбор списка корректных языков
valid_languages = languages_prepared.select("lang_clean").rdd.flatMap(lambda x: x).collect()

print("Корректные языки программирования (первые 100):")
print(valid_languages[:100])

Корректные языки программирования (первые 100):
['a# .net', 'a#', 'a-0 system', 'a+', 'a++', 'abap', 'abc', 'abc algol', 'abset', 'absys', 'acc', 'accent', 'ace dasl', 'acl2', 'act-iii', 'action!', 'actionscript', 'ada', 'adenine', 'agda', 'agilent vee', 'agora', 'aimms', 'alef', 'alf', 'algol 58', 'algol 60', 'algol 68', 'algol w', 'alice', 'alma-0', 'ambienttalk', 'amiga e', 'amos', 'ampl', 'apex', 'apl', "app inventor for android's visual block language", 'applescript', 'arc', 'arexx', 'argus', 'aspectj', 'assembly language', 'ats', 'ateji px', 'autohotkey', 'autocoder', 'autoit', 'autolisp / visual lisp', 'averest', 'awk', 'axum', 'b', 'babbage', 'bash', 'basic', 'bc', 'bcpl', 'beanshell', 'batch', 'bertrand', 'beta', 'bigwig', 'bistro', 'bitc', 'bliss', 'blockly', 'bloop', 'blue', 'boo', 'boomerang', 'bourne shell', 'bash and', 'ksh )', 'brew', 'bpel', 'c', 'c--', 'c++', 'c#', 'c/al', 'caché objectscript', 'c shell', 'caml', 'cayenne', 'cduce', 'cecil', 'cel', 'cesil', 'ceylon', '

In [61]:
# Удаляем теги, которые не являются языками программирования
posts_cleaned = posts_exploded.filter(
    lower(trim(col("Tag"))).isin(valid_languages)
)
print("Посты после удаления лишних тегов:")
posts_cleaned.show(100, truncate=False)

Посты после удаления лишних тегов:
+----+-----------+
|Year|Tag        |
+----+-----------+
|2010|c++        |
|2010|java       |
|2010|php        |
|2010|c#         |
|2010|c#         |
|2010|ruby       |
|2010|c          |
|2010|c#         |
|2010|c#         |
|2010|php        |
|2010|python     |
|2010|javascript |
|2010|applescript|
|2010|c#         |
|2010|php        |
|2010|c++        |
|2010|php        |
|2010|c#         |
|2010|javascript |
|2010|sed        |
|2010|python     |
|2010|c#         |
|2010|c#         |
|2010|java       |
|2010|c++        |
|2010|ruby       |
|2010|objective-c|
|2010|javascript |
|2010|r          |
|2010|c++        |
|2010|php        |
|2010|javascript |
|2010|python     |
|2010|php        |
|2010|c#         |
|2010|c++        |
|2010|c#         |
|2010|c          |
|2010|c#         |
|2010|c++        |
|2010|c          |
|2011|c          |
|2011|objective-c|
|2011|c#         |
|2011|c          |
|2011|javascript |
|2011|java       |
|2011|c#       